# Automatyka Pojazdowa - laboratorium 1

Celem tego laboratorium jest zapoznanie z otwartoźródłowym środowiskiem [CARLA](https://carla.org/), służącym do symulacji scenariuszy ruchu pojazdów autonomicznych. W trakcie laboratoriów będziemy korzystać ze środowiska CARLA 0.9.15.

Środowisko to pozwala na symulację end-to-end planowania, sterowania, percepcji oraz koordynacji pojazdów w kontrolowanym środowisku, które pozwala na dowolną konfigurację warunków pogodowych, ruchu drogowego czy kształtu środowiska (jezdnia, pasy, przeszkody, otoczenie).

## Pobieranie plików
W celu pobrania niezbędnych do realizacji ćwiczenia plików należy z materiałów dodatkowych pobrać i zapisać skrypt `clone_carla.sh`. Następnie należy wykonać następujące polecenia w oknie terminala:
```
chmod +x clone_carla.sh
./clone_carla.sh
cd AP-Laboratories/CARLA/
```

**Uwaga!** Wszystkie komendy wykonujemy w katalogu `AP-Laboratories/CARLA/`

## Uruchomienie środowiska CARLA

W katalogu `AP-Laboratories/CARLA/` wewnątrz repozytorium należy wykonać komendę `docker compose up`, która uruchamia 2 serwisy: carla oraz carla-viz. Pierwszy z nich to symulator CARLA wraz z GUI, które stanie się widoczne na ekranie. Drugi z nich to narzędzie pozwalające na wizualizację stanu symulacji w przeglądarce.

W razie błędu mówiącego o tym, że nazwa kontenera jest już zajęta, należy usunąć istniejące kontenery: `docker container rm -f carla_sim carla_viz`

## Uruchomienie CarlaViz
**CarlaViz** to narzędzie webowe służące do wizualizacji symulacji CARLA w czasie rzeczywistym. Umożliwia obserwację symulacji z perspektywy przeglądarki internetowej, niezależnie od głównego okna symulatora.

Aplikacja wyświetla mapę 2D/3D z pozycjami wszystkich aktorów (pojazdów, pieszych, sensorów) oraz pozwala na interaktywną nawigację po scenie. CarlaViz automatycznie łączy się z serwerem CARLA i prezentuje dane telemetryczne pojazdu ego, takie jak prędkość, przyspieszenie czy trajektoria.

Narzędzie jest szczególnie przydatne podczas developmentu algorytmów autonomicznej jazdy, gdyż zapewnia dodatkowy widok "z lotu ptaka" bez obciążania głównego wątku symulacji. CarlaViz obsługuje również wizualizację danych z sensorów, punktów orientacyjnych (waypoints) oraz debug shapes rysowanych przez API CARLA.

CarlaViz uruchamiamy wpisując w przeglądarkę adres: [`http://localhost:8080`](http://localhost:8080).

## Przygotowanie środowiska deweloperskiego

Kolejnym krokiem jest przygotowanie środowiska wirtualnego (tzw. virtual environment) Python. Aby uprościć instalację, na potrzeby tego laboratorium zostanie wykorzystane narzędzie Anaconda. Należy uruchomić na komputerze komendę:

`conda env list`

która powinna spowodować wylistowanie dostępnych środowisk na kształt poniższego:

```bash
lpa8@lpa8:~/ws$ conda env list
# conda environments:
#
base                     /home/lpa8/miniconda3
aad                      /home/lpa8/miniconda3/envs/aad
ap-carla-venv            *  /home/lpa8/miniconda3/envs/carla-venv
```

W razie, gdyby środowisko carla-venv istniało (jak w przykładzie powyżej), należy pominąć poniższą komendę; w razie jego braku - należy je stworzyć: `conda create -n ap-carla-venv python=3.8.20`.

Następnie powinniśmy zobaczyć znak zachęty z informacją, iż środowisko jest aktywne:
```bash
(ap-carla-venv) lpa8@lpa8:~/ws$
```

Prace w ramach tej serii laboratoriów należy wykonywać w tym środowisku.

Środowisko można opuścić za pomocą: `conda deactivate` lub aktywować za pomocą: `conda activate ap-carla-venv`.

**Zainstalowanie zależności:** `pip install -r requirements.txt` (w katalogu `CARLA/` wewnątrz repozytorium).

## Przygotowanie kernela Jupyter w środowisku condy

Następnie należy utworzyć konfigurację kernela Jupyter, który będzie korzystał z utworzonego powyżej środowiska wirtualnego condy:
```bash
python -m ipykernel install --user --name=ap-carla-venv --display-name "Python (ap-carla-venv)"
```

⚠️ **Krytyczne jest** ⚠️, aby wybrać to środowisko w Visual Studio Code. Można to zrobić poprzez aktywowanie okna wyboru klikając w prawym górnym rogu przycisk "Select Kernel", następnie "Jupyter Kernel..." i ostatecznie utworzoną przez nas poprzednio konfigurację "Python (ap-carla-venv)".

## Nawiązanie połączenia z CARLA

Aby połączyć się z symulatorem CARLA z API w Pythonie, należy stworzyć instancję `carla.Client`, wskazując adres hosta oraz port, na którym nasłuchuje serwer CARLA. Wykonanie poniższej komórki powinno zakończyć się sukcesem (wypisanie instancji klasy `World`, brak błędu), jeżeli dotychczasowe instrukcje zostały wykonane poprawnie. 

In [ ]:
import carla
from time import sleep

client = carla.Client('localhost', 2000)
client.set_timeout(10.0) # ustawienie czasu odpowiedzi serwera (w sek.) - jeżeli CARLA nie odpowie w tym czasie na dowolną z komend, to zostanie zwrócony wyjątek

world = client.get_world()
assert(world is not None)

# Ustaw synchroniczny tryb
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.05 # 20 FPS
world.apply_settings(settings)

print(world)

## Mapy

Symulacja w CARLA opiera się na koncepcie mapy. Mapa jest definicją m. in. otoczenia, obiektów oraz dróg. Możliwe jest zarówno listowanie dostępnych map, dodawanie nowych oraz ustawianie konkretnej mapy.

In [ ]:
# Pobranie dostępnych map
available_maps = client.get_available_maps()
print(available_maps)

# Załadowanie konkretnej mapy
client.load_world('Town04_Opt')
world = client.get_world()

# Ustaw synchroniczny tryb
settings = world.get_settings()
settings.synchronous_mode = True
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

['/Game/Carla/Maps/Town01', '/Game/Carla/Maps/Town01_Opt', '/Game/Carla/Maps/Town02', '/Game/Carla/Maps/Town02_Opt', '/Game/Carla/Maps/Town03', '/Game/Carla/Maps/Town03_Opt', '/Game/Carla/Maps/Town04', '/Game/Carla/Maps/Town04_Opt', '/Game/Carla/Maps/Town05', '/Game/Carla/Maps/Town05_Opt', '/Game/Carla/Maps/Town10HD', '/Game/Carla/Maps/Town10HD_Opt']


## Odczytywanie i filtrowanie listy aktorów

API umożliwia odczyt aktorów przez metodę klasy `World`: `get_actors()`, która zwraca listę **istniejących obiektów (aktorów) już obecnych w symulacji** - to są konkretne instancje pojazdów, pieszych, sensorów itp., które zostały już utworzone i działają w świecie CARLA. Rezultat jej działania jest iterowalny, ale też pozwala na filtrowanie przez wywołanie metody `.filter('...')`. Argumentem jest wyrażenie przypominające [glob](https://en.wikipedia.org/wiki/Glob_(programming)):

In [ ]:
# Pobranie listy wszystkich aktorów w symulacji
actors = world.get_actors()

# Wyświetlenie pierwszych 10 informacji o wszystkich aktorach
for vehicle in list(actors)[:10]:
    print(f'Aktor: {vehicle.type_id}, ID: {vehicle.id}')

# Filtrowanie obiektów typu traffic
for vehicle in list(actors.filter('traffic.*'))[:10]:
    print(f'Encja traffic: {vehicle.type_id}, ID: {vehicle.id}')

## Dodawanie pojazdu do symulacji
W symulatorze CARLA mamy zdefiniowanych prawie 40 różnych modeli samochodów, dla których możemy dodatkowo konfigurować różne inne parametry. Chcąc dodać pojazd do symulacji musimy wybrać "projekt" naszego samochodu. Listę wszystkich dostępnych "projektów" samochodów możemy odczytać metodą `get_blueprint_library()` klasy `World` w połączeniu z metodą `.filter('...')`

In [ ]:
import random

available_vehicles = world.get_blueprint_library().filter('vehicle.*')
print(f'Liczba dostępnych typów pojazdów: {len(available_vehicles)}')

# TODO odczytaj listę wszystkich modeli pojazdów marki ford i zapisz ją w zmiennej ford_vehicles
# https://carla.readthedocs.io/en/latest/python_api/#carla.BlueprintLibrary.filter
ford_vehicles = ...
print(f'Liczba modeli pojazdów marki Ford: {len(ford_vehicles)}')

# TODO wybierz losowy model spośród pojazdów marki ford i zapisz go w zmiennej ford_vehicle   
# wykrzystaj funkcje random.choice()
# https://carla.readthedocs.io/en/latest/python_api/#carla.BlueprintLibrary.filter
# https://docs.python.org/3/library/random.html#random.choice
ford_vehicle = random.choice(...)

Samochód a także innych aktorów symulacji umieszczamy na wybranej mapie podając współrzędnie miejsca. Symulator CARLA dla każdej z mapy definiuje listę predefiniowanych punktów w których można ich umieszczać. Listę predefiniowanych współrzędnych tzw. "spawn_points" można odczytać za pomocą metody `.get_spawn_points()`, klasy, `carla.Map()`

In [ ]:
# Odczytanie listy wszystkich spawn_points na mapie
spawn_points = world.get_map().get_spawn_points()
print(f'Liczba punktów spawnowania na mapie: {len(spawn_points)}')

# TODO wybierz losowo jeden z punktów spawnowania i wypisz jego współrzędne 3D (X, Y, Z)
random_spawn_point = ...
print(f'Współrzędne losowo wybranego punktu spawnowania: X={...}, Y={...}, Z={...}')

Przed dodadaniem pojazdu do symulacji warto ustawić mu atrybut 'ego'.
**Ego vehicle** to **główny pojazd** w symulacji - ten, którym sterujemy lub który obserwujemy. W kontekście autonomicznych pojazdów to "nasz" pojazd, z którego perspektywy działa system. Pojazd ego w symulacji ma dodatkowe funkcje/właściwości:
1. **Identyfikacja** - wyróżnia główny pojazd spośród innych (NPC, traffic)
2. **CarlaViz** - narzędzie wyświetla dodatkowe wskaźniki tylko dla ego (jak widzisz w komentarzu w linii 161: "w CarlaVIZ pojawią się wskaźniki, np. przyspieszenia")
3. **Sensory** - zazwyczaj sensory (kamery, lidary) montuje się na ego vehicle
4. **Algorytmy** - systemy percepcji i planowania działają z perspektywy ego

W symulacji może być wiele pojazdów, ale zazwyczaj jeden jest oznaczony jako **ego** - ten, który jest celem naszej analizy lub sterowania.

Do ustawienia właściowci **ego** służy metoda `set_attribute('role_name', 'ego')` wywoływana na obiekcie pojazdu.

In [ ]:
ford_vehicle.set_attribute('role_name', 'ego')

Jeśli mamy wczytany projekt samochodu i wybrany punkt na mapie, możemy  spróbować dodać go do symulacji. W tym celu korzystamy z metody `spawn_actor(vehicle, spawn_point)` obiektu klasy `World`.

In [ ]:
# Dodaje pojazd ford w losowym punkcie random_spawn_point
vehicle = world.spawn_actor(ford_vehicle, random_spawn_point)

Zmienna `vehicle` wskazuje na obiekt naszego pojazdu, za jej pomocą możemy kontrolować zachowanie się naszego pojazdu, odczytywać jego parametry oraz dane z sensorów pojazdu.

TODO: otwórz w przeglądarce adres: `http://localhost:8080`.

Korzystając z metod `.get_actors()` i `.filter(...)` odczytaj i wyświetl  wszystkie istniejące pojazdy w symulacji. 

In [ ]:
# TODO odczytaj ponownie listę wszystkich pojazdów w symulacji i wypisz ich typy oraz ID
for ... :  
    print(f'Pojazd: {vehicle.type_id}, ID: {vehicle.id}')

## Ustawienie pozycji spektatora

Spektator w CARLA to wirtualna kamera/obserwator, która kontroluje, co widzisz w oknie symulatora.

**Ważne:** Spektator to tylko kamera - nie wpływa na fizykę symulacji, nie ma kolizji, może przenikać przez obiekty.


In [ ]:
def set_spectator_for_vehicle(world, vehicle):
    """Positions the spectator camera behind the specified vehicle."""
    spectator = world.get_spectator()
    transform = vehicle.get_transform()
    # Get the forward vector to calculate the offset relative to the vehicle's orientation
    forward_vector = transform.get_forward_vector()

    # Calculate offset: -15 meters back, 6 meters up
    offset = carla.Location(x=-15 * forward_vector.x, y=-15 * forward_vector.y, z=6)
    # Calculate the new spectator transform
    spectator_transform = carla.Transform(
        transform.location + offset,
        # Set rotation: slightly looking down (-20 pitch), same yaw as vehicle
        carla.Rotation(pitch=-20, yaw=transform.rotation.yaw, roll=transform.rotation.roll)
    )
    # Wrap the set_transform call in a try-except block for robustness
    try:
        spectator.set_transform(spectator_transform)
        print("Spectator camera positioned behind the vehicle.")
    except Exception as e:
        print(f"Error setting spectator transform: {e}")

In [ ]:
set_spectator_for_vehicle(world, vehicle)

# TODO zmodyfikuj funkcję set_spectator_for_vehicle tak, aby pozycję kamery (wysokość, odległość za pojazdem) można było przekazać jako parametry funkcji
# TODO umieść kamerę obserwatora na miejscu kierowcy pojazdu (tzw. cockpit view)     

## Autopilot
W symulatorze CARLA można dla **dowolnego** pojazdu włączyć tryb jazdy autonomicznej. Aby to zrobić należy użyć metody `.set_autopilot(True)` na obiekcie klasy `Vehicle`

In [ ]:
vehicle.set_autopilot(True)  # Włączenie autopilota dla pojazdu

Zaobserwuj jazdę samochodu w programie CarlaViz.

Analogicznie, wyłączamy tryb jazdy autonomicznej dla danego pojazdu wywołując metodę `.set_autopilot(False)` na obiekcie klasy `Vehicle`.

In [ ]:
vehicle.set_autopilot(False)  # Włączenie autopilota dla pojazdu 

## Dodanie sensora (kamery) do pojazdu
W CARLA kamerę dodaje się jako sensor przyłączony do pojazdu przez metodę `spawn_actor()` z odpowiednim parametrem `attach_to`. Najpierw wybieramy rodzaj sensora (kamera) metodą `world.get_blueprint_library().find('sensor.camera.rgb')`. Następnym krokiem jest ustawienie miejsca, w którym kamera jest zamontowana (`carla.Transform`). Na końcu trzeba dodać utworzony sensor (kamerę) do wybranego pojazdu (np. ego) za pomocą `world.spawn_actor()`.
Przykładowy kod dodający kamerę RGB zamieszczony jest poniżej. Po dodaniu kamery zaobserwuj jej działanie w programie CarlaViz.

In [ ]:
# 1. Blueprint kamery
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')
camera_bp.set_attribute('fov', '110')

# 2. Pozycja montażu (widok z kokpitu)
camera_transform = carla.Transform(
    carla.Location(x=2.5, y=0.0, z=1.4),
    carla.Rotation(pitch=0, yaw=0, roll=0)
)

# 3. Spawn kamery
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

print(f'Kamera dodana do pojazdu: {camera.type_id}')

## Śledzenie ruchu pojazdu w symulatorze
Aby śledzić ruch pojazdu w symulatorze, musimy aktualizować pozycję spektatora w pętli, synchronizując ją z pozycją poruszającego się pojazdu.

In [ ]:
try:
    while True:
        world.tick()  # Krok symulacji
        set_spectator_for_vehicle(world, vehicle)
except KeyboardInterrupt:
    print("Przerwano śledzenie")
finally:
    vehicle.set_autopilot(False)

# Zadanie do wykonania
Dodaj 20 innych pojazdów, dla każdego z pojazdów ustaw tryb jazdy autonomicznej. Zaobserwuj ruch dodanych pojazdów z widoku z "góry" - kamera obserwatora nie podąża za pojazdem ego.

In [ ]:
# Usunięcie istniejących pojazdów
for vehicle in world.get_actors().filter('vehicle.*'):  
    vehicle.destroy()
    
# TODO
...

## Sterowanie ręczne pojazdem

Klasa `carla.VehicleControl` to klasa definiująca polecenia sterowania pojazdem. Reprezentuje stan "wirtualnego kierowcy" - pozycje pedałów i kierownicy.
```
carla.VehicleControl(
    throttle=0.0,      # Gaz: 0.0-1.0 (0%=brak, 1.0=100%)
    steer=0.0,         # Kierownica: -1.0 do 1.0 (-1=maksymalnie w lewo, +1=maksymalnie w prawo, 0=prosto)
    brake=0.0,         # Hamulec: 0.0-1.0 (0%=brak, 1.0=100%)
    hand_brake=False,  # Hamulec ręczny: True/False
    reverse=False,     # Bieg wsteczny: True/False
    manual_gear_shift=False,  # Ręczna zmiana biegów: True/False
    gear=0             # Numer biegu (jeśli manual_gear_shift=True)
)
```
Podstawowe użycie:
1. Jazda prosto z pełnym gazem:
```
control = carla.VehicleControl(throttle=1.0, steer=0.0)
vehicle.apply_control(control)
sleep(2)
```

2. Skręt w lewo z połową gazu:
```
control = carla.VehicleControl(throttle=0.5, steer=-0.5)
vehicle.apply_control(control)
sleep(2)
```
3. Hamowanie:
```
control = carla.VehicleControl(throttle=0.0, brake=1.0)
vehicle.apply_control(control)
sleep(2)
```

4. Cofanie:
```
control = carla.VehicleControl(throttle=0.5, reverse=True)
vehicle.apply_control(control)
sleep(2)
```

5. Hamulec ręczny:
```
control = carla.VehicleControl(hand_brake=True)
vehicle.apply_control(control)
sleep(2)
```

6. Zatrzymanie
```
vehicle.apply_control(carla.VehicleControl())
```

**Uwaga!**
* Wartości nie są trwałe - musisz ciągle wysyłać komendy w pętli
* `throttle` i `brake` się wykluczają - używaj jednego naraz
* W trybie synchronicznym (world.tick()) sterowanie jest bardziej precyzyjne
* Autopilot (`set_autopilot(True)`) przejmuje kontrolę - wyłącz go przed ręcznym sterowaniem


In [ ]:
# Usunięcie istniejących pojazdów
for vehicle in world.get_actors().filter('vehicle.*'):  
    vehicle.destroy()

# TODO: dodaj pojazd do symulacji
# https://carla.readthedocs.io/en/latest/python_api/#carla.World.spawn_actor
vehicle = ...

# Ustawienie ręcznego sterowania
vehicle.apply_control(carla.VehicleControl(throttle=1, steer=0.0))  # Przyspieszenie 100%, brak skrętu

# TODO: przez 10s jedź, potem zatrzymaj pojazd
...

# https://carla.readthedocs.io/en/latest/python_api/#carla.VehicleControl.brake
... # ustaw hamowanie na 100%

## Interaktywne sterowanie

Zrealizuj interaktywne sterowanie w pliku [`manual-control.py`](./manual-control.py).

Uruchom skrypt korzystając z `python manual-control.py`.